In [1]:
from utils import *
import array 

from pydub import AudioSegment
import tensorflow as tf

from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, Flatten, GlobalMaxPooling1D, MaxPooling2D, Activation, BatchNormalization, GlobalAveragePooling2D, GlobalMaxPool2D, concatenate, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.python.keras.utils import to_categorical

from keras.layers import Input, GRU, RepeatVector, BatchNormalization, TimeDistributed, Conv1D
from keras.layers import GlobalAveragePooling1D, LSTM, MaxPooling1D, CuDNNLSTM, Bidirectional
from keras import backend as K
from keras.layers import  Conv2D, MaxPooling2D, UpSampling2D, Lambda, Reshape
import keras
from keras.layers import AveragePooling1D

from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numba
from keras import losses
import dask.dataframe as dd
from dask.multiprocessing import get
from keras.layers import Embedding, Flatten, Input, merge

%matplotlib inline

Using TensorFlow backend.


In [10]:
POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}
len(id2name)

12

#### create validation dataframe

In [4]:
train_df = pickle.load( open("cache/train_df.pik","rb"))
valid_df = pickle.load( open("cache/valid_df.pik","rb"))
silent_df = pickle.load(open("cache/silent_df.pik","rb"))

unknown_df = pickle.load(open("cache/unknown_df.pik","rb"))

In [5]:
# train_df["label_user"] = train_df["label"]+train_df["user_id"]
# valid_df["label_user"] = valid_df["label"]+valid_df["user_id"]

In [71]:
silent_df["label_id"] = name2id["silence"]

In [76]:
        this_train = train_df.groupby('label_id').apply(lambda x: x.sample(n = 2000))
        extra_data_size = int(this_train.shape[0]* 0.1)
        this_train = pd.concat([silent_df.sample(extra_data_size),
                                this_train,
                                unknown_df.sample(extra_data_size*2)],axis=0 )
        
        this_train.reset_index(drop=True,inplace=True)

In [78]:
this_train.label_id.value_counts()

11    4000
10    2000
9     2000
8     2000
7     2000
6     2000
5     2000
4     2000
3     2000
2     2000
1     2000
0     2000
Name: label_id, dtype: int64

In [142]:
def train_generator(train_batch_size):
    
    

    while True:
        
        this_train = train_df.groupby('label_id').apply(lambda x: x.sample(n = 2000))
        extra_data_size = int(this_train.shape[0]* 0.1)
        this_train = pd.concat([silent_df.sample(extra_data_size),
                                this_train,
                                unknown_df.sample(extra_data_size*2)],axis=0 )
        
        this_train.reset_index(drop=True,inplace=True)
        
        
        x_batch_A_1 = []
        x_batch_A_2 = []
        x_batch_B = []
        y_batch = []
            
        for i in range(train_batch_size):


            #choose label A, and something else (B)
            first_label = np.random.randint(0,len(id2name))
            second_label = np.random.choice([i for i in list(id2name.keys()) if i !=first_label ]) #todo:hard negative

            
            
            #choose a random label A
            similars = this_train[this_train.label_id == first_label]
            #choose 2 distinct utterances from label A 
            sim_1, sim_2 = np.random.choice(range(len(similars)),2,replace=False)
            A_1 , A_2 = similars.raw.iloc[sim_1], similars.raw.iloc[sim_2] 

            # choose a different utterance from second label B
            diffs = this_train[this_train.label_id == second_label]
            B = diffs.raw.iloc[np.random.choice(range(len(diffs)))]


            # create triplet
            x_batch_A_1.append(A_1.T)
            x_batch_A_2.append(A_2.T)
            x_batch_B.append(B.T)
#                 x_batch.append([A_1.T,A_2.T,B.T])
            y_batch.append(similars.label_id.values[0])

        x_batch_A_1 = 1.- np.array(x_batch_A_1)/-80.
        x_batch_A_2 = 1.- np.array(x_batch_A_2)/-80.
        x_batch_B = 1.- np.array(x_batch_B)/-80.

#         y_batch = np.ones((train_batch_size)) 
        y_batch = to_categorical(y_batch, num_classes = len(POSSIBLE_LABELS))
    
        yield [x_batch_A_1,x_batch_A_2,x_batch_B], [np.ones((train_batch_size)) ,y_batch]# [y_batch,y_batch,y_batch]

In [143]:
t = train_generator(64)
%time tt = next(t)[0]
tt[0].mean(), tt[1].mean(), tt[2].mean(), 

Wall time: 216 ms


(0.36405783023470067, 0.37263880476216477, 0.29780967434027689)

In [144]:

tt[1].shape

(64, 32, 128)

In [145]:
def valid_generator(train_batch_size):

    while True:
        
        this_train = valid_df
        this_train.reset_index(drop=True,inplace=True)
        
        
        x_batch_A_1 = []
        x_batch_A_2 = []
        x_batch_B = []
        y_batch = []

    
            
        for i in range(train_batch_size):


            #choose label A, and something else (B)
            first_label = np.random.randint(0,len(id2name))
            second_label = np.random.choice([i for i in list(id2name.keys()) if i !=first_label ]) #todo:hard negative

            
            
            #choose a random label A
            similars = this_train[this_train.label_id == first_label]
            #choose 2 distinct utterances from label A 
            sim_1, sim_2 = np.random.choice(range(len(similars)),2,replace=False)
            A_1 , A_2 = similars.raw.iloc[sim_1], similars.raw.iloc[sim_2] 

            # choose a different utterance from second label B
            diffs = this_train[this_train.label_id == second_label]
            B = diffs.raw.iloc[np.random.choice(range(len(diffs)))]


            # create triplet
            x_batch_A_1.append(A_1.T)
            x_batch_A_2.append(A_2.T)
            x_batch_B.append(B.T)
#                 x_batch.append([A_1.T,A_2.T,B.T])
            y_batch.append(similars.label_id.values[0])

        x_batch_A_1 = 1.- np.array(x_batch_A_1)/-80.
        x_batch_A_2 = 1.- np.array(x_batch_A_2)/-80.
        x_batch_B = 1.- np.array(x_batch_B)/-80.

#         y_batch = np.ones((train_batch_size)) 
        y_batch = to_categorical(y_batch, num_classes = len(POSSIBLE_LABELS))
    
        yield [x_batch_A_1,x_batch_A_2,x_batch_B], [np.ones((train_batch_size)) ,y_batch]# [y_batch,y_batch,y_batch]

In [146]:
v = valid_generator(64)


In [147]:
%time next(v)[0][0].shape

Wall time: 132 ms


(64, 32, 128)

In [151]:
X_1,X_2,X_3, y_triplet,y_target = [], [],[], [],[]

for i in range( 150):
    [x1,x2,x3], [y1,y2] = next(v)
    X_1.append(x1) 
    X_2.append(x2) 
    X_3.append(x3) 
    
    y_triplet.append(y1)
    y_target.append(y2)

In [154]:
np.hstack(y_triplet).shape

(9600,)

In [155]:
X_1 = np.vstack(X_1)
X_2 = np.vstack(X_2)
X_3 = np.vstack(X_3)
y_target = np.vstack(y_target)
y_triplet = np.hstack(y_triplet)

In [157]:
X_1.shape, y_triplet.shape

((9600, 32, 128), (9600,))

### Model


In [119]:
def batch_relu(x):
    x = BatchNormalization()(x)    
    x = Activation('relu')(x)
    
    return x 

In [133]:
timesteps, input_dim , latent_dim = 32,128, 128
p = .0

In [134]:


# def triplet_loss(y_true, y_pred):
#     mse1 = losses.mean_squared_error(y_pred[0],y_pred[1]) #K.sqrt(y_pred[0] - y_pred[1]).mean(axis=-1)
#     mse2 = losses.mean_squared_error(y_pred[0],y_pred[2])#K.sqr(y_pred[0] - y_pred[2]).mean(axis=-1)
    
#     return K.maximum(0., mse1 - mse2 + 1) - y_true[0]*0


# def triplet_loss( y_true, y_pred):

#     embeddings = K.reshape(y_pred, (-1, 3, latent_dim))

#     positive_distance = K.mean(K.square(embeddings[:,0] - embeddings[:,1]),axis=-1)
#     negative_distance = K.mean(K.square(embeddings[:,0] - embeddings[:,2]),axis=-1)
#     return K.mean(K.maximum(0.0, positive_distance - negative_distance + 1.0))

In [135]:
def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)



def bpr_triplet_loss(X):

    positive_item_latent, negative_item_latent, user_latent = X

    # BPR loss
    loss = 1.0 - K.sigmoid(
        K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True))

    return loss



In [136]:

def embedder(filter_size=2):
    v_input = Input(shape=( input_dim,timesteps))
    x = BatchNormalization()(v_input)
    

    
    x = Conv1D(64,filter_size,padding='same')(x)
    x = batch_relu(x)
    x = Conv1D(64,filter_size,padding='same')(x)
    x = batch_relu(x)
    
    
    x  = MaxPooling1D(2)(x) 
    
    x = Conv1D(128,filter_size,padding='same')(x)
    x = batch_relu(x)
    x = Conv1D(128,filter_size,padding='same')(x)
    x = batch_relu(x)

    x  = AveragePooling1D(2)(x)   
    
#     x = Bidirectional(CuDNNLSTM(128,return_sequences=False))(x)
#     x = BatchNormalization()(x)
#     x = Dense(128,activation='relu')(x)
    
    x = GlobalMaxPooling1D()(x)
    
    
    
    
    
#     x = BatchNormalization()(v_input)
    
#     x = Conv1D(64,filter_size,padding='same')(x)
#     x = batch_relu(x)
#     x = Conv1D(64,filter_size,padding='same')(x)
#     x = batch_relu(x)
    
#     x = Conv1D(128,3,padding='same')(x)
#     x = batch_relu(x)
#     x = Conv1D(128,3,padding='same')(x)
#     x = batch_relu(x)
       
#     x = MaxPooling1D()(x)
    
#     x = Conv1D(256,3,padding='same')(x)
#     x = batch_relu(x)
#     x = Bidirectional(CuDNNLSTM(128,return_sequences=False))(x)

    return Model(v_input, x)

In [137]:
from keras.layers import *

In [138]:


xA1 = Input(shape=(timesteps, input_dim)) #1 channel, 99 time, 161 freqs # S : np.ndarray [shape=(n_mels, t)]
xA2 = Input(shape=(timesteps, input_dim)) #1 channel, 99 time, 161 freqs # S : np.ndarray [shape=(n_mels, t)]
xB = Input(shape=(timesteps, input_dim)) #1 channel, 99 time, 161 freqs # S : np.ndarray [shape=(n_mels, t)]

xA1_freq = Reshape((input_dim, timesteps))(xA1)
xA2_freq = Reshape((input_dim, timesteps))(xA2)
xB_freq = Reshape((input_dim, timesteps))(xB)



embedding = embedder(5)

xA1_freq = embedding(xA1_freq)
xA2_freq = embedding(xA2_freq)
xB_freq = embedding(xB_freq)

triplet_loss_output = merge(
    [xA1_freq, xB_freq, xA2_freq],
    mode=bpr_triplet_loss,
    name='triplet_loss_output',
    output_shape=(1,))


x = add([xA1_freq,xA2_freq])
x = Dense(128, activation = 'relu')(x) 
x = Dropout(0.3)(x)
x = Dense(len(POSSIBLE_LABELS), activation = 'softmax', name='targets')(x)


model = Model(
        inputs=[xA1, xA2, xB],
        outputs=[triplet_loss_output,x]) #concatenate([xA1_freq,xA2_freq,xB_freq],axis=0)
        
model.compile(Adam(lr=1e-3),
              loss={'targets': 'categorical_crossentropy', 'triplet_loss_output': identity_loss},
              loss_weights={'targets': 1., 'triplet_loss_output': 0.5})


C:\Users\Armin\Anaconda2\envs\cx\lib\site-packages\ipykernel_launcher.py:23: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Users\Armin\Anaconda2\envs\cx\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [139]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 32, 128)      0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 32, 128)      0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 32, 128)      0                                            
__________________________________________________________________________________________________
reshape_13 (Reshape)            (None, 128, 32)      0           input_17[0][0]                   
__________________________________________________________________________________________________
reshape_15

In [158]:
exp_name = "triplet_loss_starter" 
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1),
             
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=0.01,
                              min_lr=1e-5),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/{}.hdf5'.format(exp_name),
                             save_best_only=True,
                             save_weights_only=True,
                             mode='min')            ]

freqconv

Epoch 15/100
905/905 [==============================] - 94s 104ms/step - loss: 0.0247 - acc: 0.0245 - val_loss: 0.0230 - val_acc: 0.0236


timeconv wlstm
Epoch 17/100
905/905 [==============================] - 84s 92ms/step - loss: 0.1163 - val_loss: 0.1131

In [52]:
# model.load_weights("weights/speaker_embedding_timeconv_wlstm_c2.hdf5")

In [159]:
batch_size = 64

history = model.fit_generator(generator=train_generator(batch_size),
                              steps_per_epoch=train_df.shape[0]//batch_size,
                              epochs=100,
                              callbacks=callbacks,
                              validation_data=([X_1,X_2,X_3],[y_triplet,y_target]))#valid_generator(batch_size),
                              #validation_steps=int(np.ceil(valid_df.shape[0]//batch_size)))

Epoch 1/100
  9/329 [..............................] - ETA: 39:47 - loss: 4.5454 - triplet_loss_output_loss: 0.4952 - targets_loss: 4.29 - ETA: 21:53 - loss: 4.1928 - triplet_loss_output_loss: 0.4471 - targets_loss: 3.96 - ETA: 15:57 - loss: 3.8784 - triplet_loss_output_loss: 0.4130 - targets_loss: 3.67 - ETA: 12:56 - loss: 3.6653 - triplet_loss_output_loss: 0.4223 - targets_loss: 3.45 - ETA: 11:07 - loss: 3.5140 - triplet_loss_output_loss: 0.4102 - targets_loss: 3.30 - ETA: 10:11 - loss: 3.3935 - triplet_loss_output_loss: 0.4261 - targets_loss: 3.18 - ETA: 9:36 - loss: 3.2792 - triplet_loss_output_loss: 0.4295 - targets_loss: 3.0644 - ETA: 9:02 - loss: 3.2052 - triplet_loss_output_loss: 0.4207 - targets_loss: 2.994 - ETA: 8:31 - loss: 3.1216 - triplet_loss_output_loss: 0.4200 - targets_loss: 2.9116

KeyboardInterrupt: 

In [53]:
embedding.save_weights("weights/speaker_embedding_t2.hdf5")

In [49]:
for l in embedding.layers:
    try:
        print np.mean(l.get_weights()[0])
    except:
        continue
    

0.821782
0.00257948
0.97807
-0.0011363
0.990386
0.00306028
0.995768
0.00208017
1.00084
-0.00353018
1.10434
0.0176663


In [51]:
for l in embedding.layers:
    try:
        print np.mean(l.get_weights()[0])
    except:
        continue
    

0.82218
0.00224655
0.978326
-0.000955906
0.990576
0.00329765
0.99577
0.00218155
1.00076
-0.00361477
1.10443
0.0177416


In [47]:
for l in embedding.layers:
    try:
        print np.mean(l.get_weights()[0])
    except:
        continue
    

1.0
-0.000116561
1.0
-0.000564231
1.0
0.00077817
1.0
-6.0897e-05
1.0
0.000275884
1.0
-0.000213044
